In [1]:
import torch
import torch.nn.functional as F

In [2]:
data = open("../names.txt","r").read().splitlines()

In [3]:
chars_feature = sorted([i+j for j in set("."+"".join(data)) for i in set("."+"".join(data))])
stoi_feature  = {s:i for i,s in enumerate(chars_feature)}

label = sorted(set("."+"".join(data)))
stoi_label = {s:i for i,s in enumerate(label)}
itos = {i:s for s,i in stoi_label.items()}

In [4]:
xs = []
ys = []
for word in data:
    chars = ["."] * 2 + list(word) + ["."]
    for ch1,ch2,ch3 in zip(chars,chars[1:],chars[2:]):
        xs.append(stoi_feature[ch1+ch2])
        ys.append(stoi_label[ch3])
xs = torch.tensor(xs)
ys = torch.tensor(ys)
len(xs)

228146

In [5]:
xenc = F.one_hot(xs,num_classes=729).float()

g = torch.Generator().manual_seed(2147483647)
W = torch.randn((729, 27), generator=g, requires_grad=True)
for _ in range(1000):

    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1,keepdims=True)
    loss = -probs[torch.arange(228146),ys].log().mean() + 0.01*(W**2).mean()

    W.grad = None
    loss.backward()

    W.data += -50 * W.grad

print(loss.item())


2.2505877017974854


In [74]:
for i in range(5):
    out = ["."]
    ix = 0
    while True:
        xenc = F.one_hot(torch.tensor([ix]), num_classes=729).float()
        logits = xenc @ W 
        counts = logits.exp()
        p = counts / counts.sum(1, keepdims=True)

        next_char = torch.multinomial(p, num_samples=1, replacement=True).item()
        out.append(itos[next_char])
        if next_char == 0:
            break
        ix = stoi_feature["".join(out[-2:])]
    print(''.join(out[1:]))

prie.
zoe.
jeslyn.
alei.
coljazia.
